In [1]:
import io
import sys
PATH = "D:/OneDrive - Universidad Tecnológica de Bolívar/Apps\MachineLearning-ISCO-A14A/"
DIR_DATA = PATH + "data/"
sys.path.append(PATH) if PATH not in list(sys.path) else None
import os
import spacy
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import svm
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import preprocessing
from nltk.tokenize import RegexpTokenizer
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import LabelEncoder 
from logic.text_processing import TextProcessing
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, ShuffleSplit
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, confusion_matrix, recall_score, log_loss
from sklearn.metrics import f1_score, accuracy_score, precision_score

In [2]:
tp = TextProcessing()
le = LabelEncoder()
nlp = spacy.load("es_core_news_sm")

In [3]:
data_train = pd.read_csv(DIR_DATA + 'tass2018_es_train.csv', sep=',')
data_train

,tweetid,user,content,date,lang,sentiment/polarity/value
0,768213876278165504,OnceBukowski,-Me caes muy bien \r\n-Tienes que jugar más pa...,2016-08-23 22:30:35,es,NONE
1,768213567418036224,anahorxn,@myendlesshazza a. que puto mal escribo\r\n\r\...,2016-08-23 22:29:21,es,N
2,768212591105703936,martitarey13,@estherct209 jajajaja la tuya y la d mucha gen...,2016-08-23 22:25:29,es,N
3,768221670255493120,endlessmilerr,Quiero mogollón a @AlbaBenito99 pero sobretodo...,2016-08-23 23:01:33,es,P
4,768221021300264964,JunoWTFL,Vale he visto la tia bebiendose su regla y me ...,2016-08-23 22:58:58,es,N
...,...,...,...,...,...,...
1003,814846333601320960,jdomperfects,Para mí mi mejor año fue el 2015,2016-12-30 14:51:19,es,P
1004,813731371076243461,GonzaloRoman_91,Hoy va a ser un dia muy largo...,2016-12-27 13:00:51,es,N
1005,818399956792905728,vaneftdallas,11:11 que no me encuentre mal,2017-01-09 10:12:09,es,NEU
1006,815715581878009858,carmen_ligero,Es que son muy bonitas jo,2017-01-02 00:25:24,es,P


In [4]:
data_test = pd.read_csv(DIR_DATA + 'tass2018_es_development.csv', sep=',')
data_test

,tweetid,user,content,date,lang,sentiment/polarity/value
0,770976639173951488,noseashetero,@noseashetero 1000/10 de verdad a ti que voy a...,2016-08-31 13:28:49,es,P
1,771092421866389508,Templelx,@piscolabisaereo @HistoriaNG @SPosteguillo las...,2016-08-31 21:08:54,es,P
2,771092111429083136,esskuu94,"Al final han sido 3h Bueno, mañana tengo fies...",2016-08-31 21:07:40,es,P
3,771092070572449796,__ariadna9,@Jorge_Ruiz14 yo no tengo tiempo para esas cos...,2016-08-31 21:07:30,es,N
4,771094192508600320,_cristtina15_,@_MissChaotic_ ves ese brillo? es un coso que ...,2016-08-31 21:15:56,es,N
...,...,...,...,...,...,...
501,757593585634213888,RubenRuCh,@Davorias Pero es que está muy rica,2016-07-25 15:09:20,es,P
502,757585725764362240,rayjaken,@narvalillo Eso es antiguo ya,2016-07-25 14:38:06,es,N
503,757097408304148480,Fuhyo__,Mi TL esta vacía a estas horas,2016-07-24 06:17:42,es,N
504,757971881077182464,xLytherinDovah,"Ya tengo uno, pero ese es más cheto @aweamasome.",2016-07-26 16:12:33,es,NEU


In [5]:
x_train = [tp.transformer(row) for row in data_train['content'].tolist()]
#y_train = le.fit_transform(data_train['sentiment/polarity/value'])
y_train = data_train['sentiment/polarity/value']
len(x_train), len(y_train)

(1008, 1008)

In [6]:
x_test = [tp.transformer(row) for row in data_test['content'].tolist()]
#y_test = le.fit_transform(data_test['sentiment/polarity/value'])
y_test = data_test['sentiment/polarity/value']
len(x_test), len(y_test)

(506, 506)

In [7]:
for row in x_train:
    doc = nlp(str(row))
    pos = [token.pos_ for token in doc]
    print(pos)

['PRON', 'VERB', 'ADV', 'ADV', 'VERB', 'SCONJ', 'VERB', 'PROPN', 'NOUN', 'ADP', 'NOUN', 'ADP', 'PROPN', 'CCONJ', 'NOUN', 'ADP', 'SCONJ', 'ADV', 'VERB', 'VERB', 'ADP', 'AUX', 'VERB', 'AUX', 'SCONJ', 'PRON', 'VERB']
['NOUN', 'ADP', 'SCONJ', 'ADJ', 'ADV', 'VERB', 'SYM', 'PRON', 'VERB', 'VERB', 'PROPN', 'AUX', 'VERB', 'ADJ', 'DET', 'NOUN', 'PROPN', 'VERB']
['PROPN', 'VERB', 'DET', 'PRON', 'CCONJ', 'DET', 'NOUN', 'DET', 'NOUN', 'ADJ', 'CCONJ', 'PRON', 'ADV', 'VERB', 'ADP', 'DET', 'NOUN', 'PRON', 'VERB']
['VERB', 'PROPN', 'ADP', 'PROPN', 'CCONJ', 'NOUN', 'ADP', 'PRON', 'ADJ', 'SCONJ', 'VERB', 'ADP', 'DET', 'PROPN']
['VERB', 'AUX', 'VERB', 'DET', 'NOUN', 'VERB', 'DET', 'NOUN', 'CCONJ', 'PRON', 'VERB', 'VERB', 'PROPN', 'PROPN']
['PROPN', 'PROPN', 'ADV', 'DET', 'PROPN', 'ADP', 'ADJ', 'ADJ', 'SCONJ', 'PRON', 'ADJ', 'PRON', 'VERB', 'VERB', 'ADV']
['PRON', 'AUX', 'VERB', 'VERB', 'ADV', 'ADV', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN', 'PRON', 'PRON', 'VERB', 'NOUN', 'CCONJ'

['NOUN', 'ADV', 'SCONJ', 'DET', 'NOUN', 'AUX', 'AUX', 'ADV', 'ADJ', 'VERB', 'ADP', 'DET', 'PRON', 'CCONJ', 'DET', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'NOUN']
['PROPN', 'SCONJ', 'ADV', 'ADJ', 'SCONJ', 'ADP', 'DET', 'NOUN', 'PRON', 'VERB', 'ADP', 'VERB', 'CCONJ', 'AUX', 'ADP', 'PRON', 'PRON', 'PROPN', 'NOUN', 'PRON', 'VERB', 'ADV', 'VERB', 'PROPN', 'VERB', 'NOUN']
['PROPN', 'ADV', 'ADJ', 'ADJ', 'CCONJ', 'AUX', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADJ', 'ADP', 'DET', 'PRON', 'NOUN', 'ADJ']
['PROPN', 'VERB', 'DET', 'NOUN', 'CCONJ', 'VERB', 'ADJ', 'ADV', 'ADP', 'PROPN', 'AUX', 'ADJ', 'PRON', 'VERB']
['ADV', 'SCONJ', 'AUX', 'ADP', 'PROPN', 'ADV', 'ADV', 'AUX', 'VERB', 'NOUN', 'VERB', 'ADJ', 'ADP', 'DET', 'NOUN', 'CCONJ', 'ADV']
['NOUN', 'PUNCT', 'PRON', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'CCONJ', 'ADP', 'SCONJ', 'AUX', 'VERB', 'ADP', 'VERB', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ']
['PROPN', 'PROPN', 'PRON', 'ADP', 'VERB', 'DET', 'NOUN', 'CCONJ', 'AUX', 'ADJ', 'PRON', 'AUX', 'ADJ', 'CCONJ

['PROPN', 'SCONJ', 'VERB', 'PRON', 'VERB', 'SCONJ', 'DET', 'ADV', 'AUX', 'ADV', 'AUX', 'DET', 'NOUN', 'CCONJ', 'ADV', 'DET', 'NOUN', 'ADV']
['AUX', 'ADJ', 'VERB', 'DET', 'NOUN', 'ADJ', 'PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADV', 'PRON', 'AUX', 'VERB', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'NOUN']
['PROPN', 'PROPN', 'DET', 'ADJ', 'AUX', 'DET', 'NOUN', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADV', 'PRON']
['DET', 'NOUN', 'ADP', 'DET', 'ADP', 'DET', 'ADJ', 'VERB', 'ADJ', 'CCONJ', 'ADP', 'NOUN', 'PRON', 'VERB', 'VERB']
['PROPN', 'PRON', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADJ', 'ADP', 'VERB']
['PROPN', 'PRON', 'ADV', 'AUX', 'PRON', 'VERB', 'SCONJ', 'VERB', 'ADP', 'DET', 'NOUN', 'ADJ', 'PRON', 'VERB', 'ADP', 'DET', 'CCONJ', 'ADJ', 'ADP', 'ADV', 'ADJ']
['AUX', 'ADJ', 'SCONJ', 'ADV', 'DET', 'DET', 'NOUN', 'ADJ', 'AUX', 'AUX', 'SCONJ', 'PRON', 'AUX', 'VERB']
['PROPN', 'PROPN', 'PROPN', 'CCONJ', 'ADV', 'PRON', 'VERB', 'ADV', 'NOUN', 'DET', 'NOUN', 'NOUN']
['ADV', 'NOUN', 'VERB

['PROPN', 'PRON', 'ADV', 'VERB', 'ADV', 'NOUN', 'CCONJ', 'AUX', 'DET', 'ADJ', 'PRON', 'VERB', 'NOUN', 'ADJ', 'CCONJ', 'ADP', 'DET', 'NOUN', 'PRON', 'VERB', 'ADP', 'SCONJ', 'AUX', 'ADV', 'ADJ']
['PROPN', 'PROPN', 'DET', 'NOUN', 'AUX', 'ADJ', 'ADP', 'DET', 'NOUN', 'PRON', 'VERB', 'NOUN', 'PRON', 'NOUN', 'SCONJ', 'AUX', 'NOUN']
['VERB', 'DET', 'NOUN', 'PROPN', 'NOUN', 'CCONJ', 'ADP', 'NOUN', 'DET', 'NOUN', 'PRON', 'VERB', 'DET', 'ADV', 'ADJ', 'ADP', 'AUX', 'ADV', 'PRON', 'PRON', 'VERB', 'SCONJ', 'VERB']
['PROPN', 'PROPN', 'PROPN', 'PROPN', 'PROPN', 'ADV', 'ADJ', 'CCONJ', 'ADV', 'VERB', 'DET', 'ADJ', 'NOUN', 'ADP', 'PROPN', 'ADP', 'PRON']
['PROPN', 'DET', 'NOUN', 'AUX', 'ADJ', 'CCONJ', 'DET', 'ADJ', 'CCONJ', 'VERB', 'CCONJ', 'PRON', 'NOUN', 'ADP', 'NOUN', 'ADP', 'VERB']
['PROPN', 'PROPN', 'PRON', 'VERB', 'ADP', 'VERB', 'CCONJ', 'ADP', 'VERB', 'ADV', 'ADJ', 'VERB', 'SCONJ', 'AUX', 'ADP', 'NOUN', 'ADP', 'VERB', 'PRON', 'ADP', 'DET', 'NOUN']
['SCONJ', 'AUX', 'SCONJ', 'VERB', 'NOUN', 'ADP', 'D

['AUX', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'ADP', 'DET', 'ADJ', 'NOUN', 'PROPN', 'ADP', 'PROPN']
['VERB', 'NUM', 'NOUN', 'ADJ', 'VERB', 'PROPN', 'ADP', 'DET', 'NOUN']
['SCONJ', 'PRON', 'VERB', 'ADP', 'ADV', 'NOUN', 'VERB', 'AUX']
['PROPN', 'VERB', 'SCONJ', 'NOUN', 'CCONJ', 'ADP', 'NOUN', 'VERB', 'DET', 'NOUN', 'PROPN', 'NOUN', 'PROPN']
['PROPN', 'AUX', 'PRON', 'PRON', 'VERB', 'NOUN', 'ADP', 'PRON', 'PRON', 'VERB', 'PRON', 'VERB', 'DET', 'NOUN', 'NOUN', 'ADJ']
['PROPN', 'NOUN', 'ADJ', 'ADJ', 'ADP', 'DET', 'NOUN', 'ADV', 'PRON', 'VERB', 'ADV', 'PRON']
['PROPN', 'SCONJ', 'AUX', 'PRON', 'CCONJ', 'ADJ', 'PRON', 'PRON', 'PRON', 'DET', 'PRON', 'VERB', 'DET', 'NOUN', 'SCONJ', 'PROPN', 'VERB']
['PRON', 'SCONJ', 'AUX', 'DET', 'NOUN', 'ADV', 'ADJ', 'CCONJ', 'PRON', 'VERB', 'ADJ', 'DET', 'NOUN', 'ADV', 'ADV']
['PROPN', 'SCONJ', 'PRON', 'SCONJ', 'AUX', 'DET', 'NOUN', 'CCONJ', 'ADV', 'VERB', 'ADP', 'VERB', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADV', 'VERB', 'DET', 'ADJ', 'NOUN', 'PRON', 'VERB', 

...

['ADV', 'PRON', 'VERB', 'ADP', 'VERB', 'PRON', 'VERB', 'ADP', 'NOUN', 'ADP', 'VERB', 'ADJ', 'ADP', 'CCONJ', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'ADJ', 'NOUN']
['PROPN', 'PROPN', 'PRON', 'VERB', 'VERB', 'ADV']
['DET', 'NOUN', 'ADJ', 'DET', 'ADJ', 'SCONJ', 'DET', 'NOUN', 'ADP', 'ADV']
['PART', 'NOUN', 'VERB', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'ADJ', 'PROPN']
['PROPN', 'PROPN', 'PROPN', 'PROPN', 'ADJ', 'ADV', 'ADV', 'AUX', 'ADJ']
['PROPN', 'PROPN', 'PRON', 'ADV', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'CCONJ', 'AUX', 'ADP', 'ADJ', 'NOUN']
['PROPN', 'AUX', 'VERB', 'VERB', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'PRON', 'ADV', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN', 'AUX', 'VERB', 'PRON', 'VERB', 'ADV']
['NOUN', 'PRON', 'NOUN', 'ADP', 'NOUN', 'ADP', 'VERB', 'ADP', 'NOUN', 'CCONJ', 'VERB', 'PRON', 'PROPN', 'PROPN', 'SCONJ', 'VERB', 'ADP', 'SCONJ', 'DET', 'NOUN', 'ADV', 'PRON', 'VERB']
['PROPN', 'VERB', 'VERB', 'DET', 'NOUN', 'ADP', 'NOUN', 'ADP', 'DET', 'NOUN', 'SCONJ', 'VERB', 'SCONJ', 'VERB', 'ADJ',

['PROPN', 'AUX', 'ADP', 'AUX', 'NOUN', 'ADV', 'NOUN', 'ADP', 'PROPN', 'DET', 'VERB', 'PRON', 'VERB', 'DET', 'ADV', 'ADJ', 'ADP', 'VERB', 'PRON', 'ADJ']
['VERB', 'ADP', 'AUX', 'DET', 'ADJ', 'NOUN', 'ADP', 'AUX', 'VERB', 'PRON', 'VERB', 'NOUN', 'ADP', 'DET', 'NOUN']
['NOUN', 'PRON', 'VERB', 'AUX', 'VERB', 'ADV', 'VERB', 'VERB', 'PRON']
['PROPN', 'PROPN', 'DET', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'ADJ', 'ADP', 'PRON', 'ADJ', 'CCONJ', 'VERB', 'SCONJ', 'VERB', 'ADP', 'DET', 'NOUN']
['SCONJ', 'VERB', 'ADV', 'VERB', 'DET', 'PROPN', 'ADV', 'AUX', 'NOUN', 'PRON', 'NOUN', 'SCONJ', 'CCONJ', 'VERB', 'NOUN', 'DET', 'PRON', 'VERB', 'ADP', 'NOUN']
['PROPN', 'PROPN', 'PRON', 'ADV', 'AUX', 'ADJ', 'ADP', 'DET', 'NOUN', 'NOUN']
['ADJ', 'NOUN', 'ADP', 'DET', 'PROPN', 'PROPN', 'ADJ', 'CCONJ', 'PROPN', 'PROPN', 'VERB']
['PROPN', 'PROPN', 'PROPN', 'PROPN', 'VERB', 'ADJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'DET', 'NOUN']
['ADV', 'PRON', 'VERB', 'NOUN', 'PRON', 'VERB', 'ADJ', 'CCONJ', 'ADV', 'PRON', 'VER

['ADJ', 'NOUN', 'VERB', 'ADP', 'NOUN', 'VERB', 'SCONJ', 'PRON', 'VERB', 'DET', 'NOUN', 'PRON', 'DET', 'NOUN', 'NOUN', 'ADP', 'ADV', 'NOUN']
['AUX', 'VERB', 'DET', 'ADJ', 'VERB', 'DET', 'ADJ', 'ADJ', 'ADP', 'NOUN', 'ADJ', 'VERB', 'ADV', 'VERB', 'NOUN']
['PROPN', 'ADV', 'ADJ', 'SCONJ', 'DET', 'NOUN', 'PRON', 'VERB', 'ADP', 'DET', 'ADJ', 'NOUN', 'ADP', 'ADP', 'NOUN']
['NOUN', 'NOUN', 'VERB', 'CCONJ', 'AUX', 'SCONJ', 'AUX', 'NOUN', 'VERB', 'NOUN', 'CCONJ', 'DET', 'NOUN', 'CCONJ', 'PRON', 'SCONJ', 'AUX', 'NOUN', 'AUX', 'ADP', 'AUX', 'DET', 'ADJ', 'PRON', 'VERB']
['PROPN', 'ADV', 'VERB', 'DET', 'NOUN', 'VERB', 'ADV', 'SCONJ', 'ADP', 'NOUN', 'PRON', 'AUX', 'VERB', 'NOUN', 'ADV', 'ADV']
['PROPN', 'SCONJ', 'VERB', 'VERB', 'CCONJ', 'VERB', 'ADJ', 'DET', 'NOUN', 'ADV', 'PRON', 'VERB', 'ADJ']
['PROPN', 'DET', 'NOUN', 'AUX', 'PRON', 'ADP', 'DET', 'NOUN', 'ADJ', 'CCONJ', 'ADP', 'PROPN', 'ADP', 'NOUN', 'PRON', 'VERB', 'NOUN']
['NOUN', 'ADJ', 'DET', 'NOUN', 'AUX', 'DET', 'NOUN', 'ADJ', 'CCONJ', 'DET',

['VERB', 'VERB', 'ADP', 'PROPN', 'NOUN', 'VERB', 'NOUN', 'CCONJ', 'NOUN', 'ADP', 'DET', 'NOUN', 'PRON', 'VERB', 'DET']
['PROPN', 'PROPN', 'ADJ', 'PRON', 'ADJ', 'ADP', 'PRON', 'ADP', 'NOUN', 'AUX', 'NOUN', 'ADJ', 'ADJ', 'DET', 'NOUN', 'AUX', 'SCONJ', 'PRON', 'VERB', 'DET', 'PROPN', 'ADJ']
['PROPN', 'VERB', 'NOUN', 'ADP', 'NOUN', 'ADP', 'NOUN', 'PRON', 'DET', 'NOUN', 'VERB', 'DET', 'PROPN']
['DET', 'NOUN', 'PRON', 'PROPN', 'PROPN', 'PRON', 'VERB', 'ADJ', 'SCONJ', 'VERB', 'AUX', 'PRON', 'ADP', 'DET', 'NOUN', 'ADJ', 'ADP', 'SCONJ', 'VERB', 'ADJ']
['ADP', 'NOUN', 'PROPN', 'SCONJ', 'PRON', 'ADV']
['PROPN', 'PRON', 'VERB', 'ADV', 'AUX', 'ADV', 'ADJ', 'PRON', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PRON', 'VERB', 'SCONJ', 'ADV', 'PRON', 'VERB', 'PROPN', 'PROPN']
['PROPN', 'PRON', 'PRON', 'CCONJ', 'ADV', 'VERB', 'DET', 'NOUN', 'ADV', 'VERB', 'ADP', 'NOUN', 'ADP', 'ADV', 'DET', 'NOUN', 'CCONJ', 'PRON', 'ADJ', 'CCONJ', 'NOUN', 'CCONJ', 'PROPN']
['PROPN', 'PRON', 'ADJ', 'AUX', 'VERB', 'ADP'

['ADJ', 'ADP', 'PRON', 'ADV', 'ADP', 'DET', 'PRON']
['AUX', 'DET', 'NOUN', 'ADV', 'ADJ']
['CCONJ', 'ADV', 'ADJ', 'SCONJ', 'ADV', 'PRON', 'PRON', 'VERB']
['DET', 'NOUN', 'PRON', 'VERB', 'ADV', 'ADJ']
['NOUN', 'CCONJ', 'CCONJ', 'SCONJ', 'AUX', 'ADV', 'ADJ']
['PROPN', 'CCONJ', 'PRON', 'ADV', 'AUX', 'NOUN']
['PROPN', 'ADV', 'ADV', 'CCONJ', 'ADV', 'ADV', 'ADV', 'PRON', 'VERB']
['PROPN', 'AUX', 'SCONJ', 'ADV', 'ADV', 'AUX', 'ADJ']
['DET', 'ADJ', 'NOUN', 'AUX', 'DET', 'PRON', 'ADV', 'PRON', 'VERB']
['DET', 'NOUN', 'AUX', 'SCONJ', 'ADV', 'VERB', 'DET', 'ADJ', 'NOUN']
['PROPN', 'ADV', 'VERB', 'PRON', 'AUX', 'ADJ']
['PROPN', 'CCONJ', 'PRON', 'NOUN', 'PRON', 'DET', 'NOUN']
['AUX', 'ADJ', 'CCONJ', 'ADV', 'ADP', 'DET', 'NOUN']
['PROPN', 'VERB', 'SCONJ', 'ADP', 'DET', 'ADJ', 'NOUN']
['NOUN', 'PRON', 'ADV', 'ADP', 'NOUN']
['NOUN', 'PRON', 'AUX', 'DET', 'NOUN']
['PROPN', 'ADV', 'ADV', 'ADV', 'ADV', 'PRON', 'VERB', 'ADP', 'DET', 'NOUN']
['CCONJ', 'SCONJ', 'AUX', 'DET', 'ADJ', 'PROPN']
['ADV', 'PRON', '

In [8]:
class POSExtraction(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, list_messages):
        try:
            result = self.get_features(list_messages)
            return result
        except Exception as e:
            print('Error transform: {0}'.format(e))
            
    def get_features(self, list_messages):
        result = {}
        i = 0 
        for row in list_messages:
            dict_pos = {}
            doc = nlp(str(row))
            pos = [token.pos_ for token in doc]
            for token in pos:
                if token not in dict_pos:
                    dict_pos[token] = 1
                else:
                    val = dict_pos[token]
                    dict_pos[token] = val + 1
            result[i] = dict_pos
            i += 1
        features = pd.DataFrame.from_dict(result, orient='index').fillna(0)
        return features.to_numpy()

In [9]:
tfidf_vectoriser = TfidfVectorizer(max_features=5000, min_df=2, max_df=0.9, ngram_range=(1,3))
bow_vector = CountVectorizer(analyzer='word', ngram_range=(1, 3))
pos_vector = POSExtraction()

In [10]:
pos_vector.transform(['Hola esto es una prueba', 'otra prueba'])

array([[1., 1., 1., 1., 1.],
       [0., 0., 0., 1., 1.]])

In [11]:
preprocessor = FeatureUnion([
    ('bow_vector', bow_vector),
    ('pos_vector', pos_vector)
])

In [12]:
preprocessor.fit(x_train)

FeatureUnion(transformer_list=[('bow_vector',
                                CountVectorizer(ngram_range=(1, 3))),
                               ('pos_vector', POSExtraction())])

In [13]:
x_train = preprocessor.transform(x_train)

In [14]:
len(x_train.toarray())

1008

In [15]:
x_test = preprocessor.transform(x_test)

In [16]:
print('**Sample train:', sorted(Counter(y_train).items()))

**Sample train: [('N', 418), ('NEU', 133), ('NONE', 139), ('P', 318)]


In [17]:
print('**Sample test:', sorted(Counter(y_test).items()))

**Sample test: [('N', 219), ('NEU', 69), ('NONE', 62), ('P', 156)]


In [18]:
k_fold = ShuffleSplit(n_splits=10, test_size=0.25, random_state=42)

In [19]:
ros_train = RandomOverSampler(random_state=1000)
x_train, y_train = ros_train.fit_resample(x_train, y_train)

In [20]:
print('**OverSample train:', sorted(Counter(y_train).items()))

**OverSample train: [('N', 418), ('NEU', 418), ('NONE', 418), ('P', 418)]


In [21]:
ros_test = RandomOverSampler(random_state=1000)
x_test, y_test = ros_test.fit_resample(x_test, y_test)

In [22]:
print('**OverSample test:', sorted(Counter(y_test).items()))

**OverSample test: [('N', 219), ('NEU', 219), ('NONE', 219), ('P', 219)]


In [23]:
clf = svm.SVC(kernel='poly', degree=3, C=1).fit(x_train, y_train)

In [24]:
accuracies_scores = []
recalls_scores = []
precisions_scores = []
f1_scores = []

In [25]:
for train_index, test_index in k_fold.split(x_train, y_train):
    data_train = x_train[train_index]
    target_train = y_train[train_index]
    
    data_test = x_train[test_index]
    target_test = y_train[test_index]

    clf.fit(data_train, target_train)
    predict = clf.predict(data_test)
    # Accuracy
    accuracy = accuracy_score(target_test, predict)
    accuracies_scores.append(accuracy)
    # Recall
    recall = recall_score(target_test, predict, average='macro')
    recalls_scores.append(recall)
    # Precision
    precision = precision_score(target_test, predict, average='weighted')
    precisions_scores.append(precision)
    # F1
    f1 = f1_score(target_test, predict, average='micro')
    f1_scores.append(f1)

In [26]:
average_recall = round(np.mean(recalls_scores) * 100, 2)
average_precision = round(np.mean(precisions_scores) * 100, 2)
average_f1 = round(np.mean(f1_scores) * 100, 2)
average_accuracy = round(np.mean(accuracies_scores) * 100, 2)

In [27]:
y_predict = []
for features in x_test:
    features = features.reshape(1, -1)
    value = clf.predict(features)[0]
    y_predict.append(value)

classification = classification_report(y_test, y_predict)
confusion = confusion_matrix(y_predict, y_test)

In [28]:
output_result = {'F1-score': average_f1, 'Accuracy': average_accuracy, 'Recall': average_recall, 
                 'Precision': average_precision, 'Classification Report\n': classification, 
                 'Confusion Matrix\n': confusion}

In [29]:
for item, val in output_result.items():
    print('{0} {1}'.format(item, val))

F1-score 66.24
Accuracy 66.24
Recall 66.94
Precision 75.68
Classification Report
               precision    recall  f1-score   support

           N       0.39      0.25      0.31       219
         NEU       0.00      0.00      0.00       219
        NONE       0.58      0.03      0.06       219
           P       0.25      0.83      0.39       219

    accuracy                           0.28       876
   macro avg       0.31      0.28      0.19       876
weighted avg       0.31      0.28      0.19       876

Confusion Matrix
 [[ 55  27  27  31]
 [  0   0   3   3]
 [  2   0   7   3]
 [162 192 182 182]]
